## 1. Write out the usage and manual testing instructions as Markdown.
- We're doing this as documentation-driven development.
- What should the user expect to see in the table after running each script?

See 'instructions.md'.

## 2. Write the get_latest_data function.
- This function should return only two columns: e.g. dates and cpi
- All other code should interact with the source data only through this function

In [7]:
import pandas as pd 
import duckdb

In [8]:
def get_latest_data(filename, pull_date):
    year, month, _ = pull_date.split('-')
    year = year[2:] 
    month = str(int(month))

    # Read data
    data = pd.read_excel(filename)

    # Generate the required column name
    cpi = f"PCPI{year}M{month}"
    
    # Extract necessary data
    data = data[['DATE', cpi]].rename(columns={'DATE': 'date', cpi: 'cpi'})
    
    # Fix the 'date' column format
    data['date'] = pd.to_datetime(data['date'].astype(str).str.replace(':', '-') + '-01')
    
    # Replace NaN values in 'cpi' column with 0
    data['cpi'] = data['cpi'].fillna(0)
    
    return data

# Load data
filename = "CPI.xlsx"
data = get_latest_data(filename, '2021-01-14')
data

,date,cpi
0,1947-01-01,21.48
1,1947-02-01,21.62
2,1947-03-01,22.00
3,1947-04-01,22.00
4,1947-05-01,21.95
...,...,...
932,2024-09-01,0.00
933,2024-10-01,0.00
934,2024-11-01,0.00
935,2024-12-01,0.00


## 3. Work through each method of data loading.
- Include the type in the scripts and table names to keep them separate — something like:
    - _append
    - _trunc
    -  _inc
- Your code should accept a pull_date parameter and load the data up to that date
- The script should be able to run multiple times without duplicating data
- For incremental: a Python script may be easier than a SQL one

### Setup Duckdb

In [9]:
# Create DuckDB database
file = 'cpi.db'
con = duckdb.connect(file)

# Add the original data to the database
cpi_df = pd.read_excel("CPI.xlsx")
con.sql("CREATE OR REPLACE TABLE cpi AS SELECT * FROM cpi_df")

# Show tables
print(con.sql('SHOW TABLES').fetchdf())
print()

# Show the original data
print(con.sql('SELECT * FROM cpi LIMIT 5').fetchdf())

# close the connection
con.close()

               name
0               cpi
1       data_append
2      data_append1
3  data_incremental

      DATE  PCPI98M11  PCPI98M12  PCPI99M1  PCPI99M2  PCPI99M3  PCPI99M4  \
0  1947:01        NaN        NaN       NaN       NaN       NaN       NaN   
1  1947:02        NaN        NaN       NaN       NaN       NaN       NaN   
2  1947:03        NaN        NaN       NaN       NaN       NaN       NaN   
3  1947:04        NaN        NaN       NaN       NaN       NaN       NaN   
4  1947:05        NaN        NaN       NaN       NaN       NaN       NaN   

   PCPI99M5  PCPI99M6  PCPI99M7  ...  PCPI24M5  PCPI24M6  PCPI24M7  PCPI24M8  \
0       NaN       NaN       NaN  ...     21.48     21.48     21.48     21.48   
1       NaN       NaN       NaN  ...     21.62     21.62     21.62     21.62   
2       NaN       NaN       NaN  ...     22.00     22.00     22.00     22.00   
3       NaN       NaN       NaN  ...     22.00     22.00     22.00     22.00   
4       NaN       NaN       NaN  ...     2

### 3-1. Append load

In [10]:
#Alexa
def append_load(con, data):
    # Create table if it does not exist (ensure this matches the actual structure of data_append)
    con.sql("""
        CREATE TABLE IF NOT EXISTS data_append1 (
            date DATE,
            cpi REAL
            -- Add other columns if necessary, or set default values
        )
    """)
    
    #appending new data
    for _, row in data.iterrows():
        date_str = row['date'].strftime('%Y-%m-%d')
        cpi_value = 'NULL' if pd.isna(row['cpi']) else row['cpi']
        con.sql(
            f"INSERT INTO data_append1 (date, cpi) VALUES ('{date_str}', {cpi_value if cpi_value != 'NULL' else 'NULL'})"
        )

# Get new data
filename = "CPI.xlsx"
new_data = get_latest_data(filename, '2022-12-14')

with duckdb.connect(file) as con:
    con.sql('BEGIN TRANSACTION')  
    append_load(con, new_data)
    con.sql('COMMIT')  
    print(con.sql('SELECT * FROM data_append1').fetchdf())



            date        cpi
0     1947-01-01  21.480000
1     1947-02-01  21.620001
2     1947-03-01  22.000000
3     1947-04-01  22.000000
4     1947-05-01  21.950001
...          ...        ...
29979 2024-09-01   0.000000
29980 2024-10-01   0.000000
29981 2024-11-01   0.000000
29982 2024-12-01   0.000000
29983 2025-01-01   0.000000

[29984 rows x 2 columns]


### 3-2. Truncate and load

In [19]:
## Giulio

### 3-3. Incremental load

In [11]:
def incremental_load(con, data):
    
    # Create table if it does not exist
    con.sql("""
        CREATE TABLE IF NOT EXISTS data_incremental (
            date DATE,
            cpi REAL
        )
    """)
    
    # Retrieve existing data
    existing_dates = con.sql('SELECT date, cpi FROM data_incremental').fetchdf()
    
    if existing_dates.empty:
        # If no existing data, insert new data directly
        for _, row in data.iterrows():
            date_str = row['date'].strftime('%Y-%m-%d')
            cpi_value = 'NULL' if pd.isna(row['cpi']) else row['cpi']
            con.sql(
                f"INSERT INTO data_incremental VALUES ('{date_str}', {cpi_value if cpi_value != 'NULL' else 'NULL'})"
                )
        return
    
    # Merge data for comparison
    merged_dates = existing_dates.merge(data[['date','cpi']], on='date', suffixes=('_old', '_new'))
    
    # Identify data that needs updating
    dates_to_remove = merged_dates[merged_dates['cpi_old'] != merged_dates['cpi_new']]['date']
    
    # Delete outdated data
    for date in dates_to_remove:
        con.sql(f"DELETE FROM data_incremental WHERE date = '{date}'")
    
    # Get the most recent date
    most_recent_date = con.sql('SELECT MAX(date) FROM data_incremental').fetchdf().values[0][0]
    
    # Filter data to insert only new entries
    if most_recent_date is not None:
        data = data[data['date'] > most_recent_date]
    
    # Insert new data
    for _, row in data.iterrows():
        date_str = row['date'].strftime('%Y-%m-%d')
        cpi_value = 'NULL' if pd.isna(row['cpi']) else row['cpi']
        con.sql(
            f"INSERT INTO data_incremental VALUES ('{date_str}', {cpi_value if cpi_value != 'NULL' else 'NULL'})"
            )

# Get new data
filename = "CPI.xlsx"
new_data = get_latest_data(filename, '2022-12-14')

with duckdb.connect(file) as con:
    con.sql('BEGIN TRANSACTION') # Start transaction
    incremental_load(con, new_data)
    con.sql('COMMIT') # Commit transaction
    print(con.sql('SELECT * FROM data_incremental').fetchdf())

           date    cpi
0    1947-03-01  22.00
1    1947-04-01  22.00
2    1948-03-01  23.50
3    1949-09-01  23.75
4    1950-10-01  24.50
...         ...    ...
1862 2024-09-01   0.00
1863 2024-10-01   0.00
1864 2024-11-01   0.00
1865 2024-12-01   0.00
1866 2025-01-01   0.00

[1867 rows x 2 columns]


## 4. On a notebook: simulate your organization running the scripts on a daily basis.
- Start from empty tables
- Loop over a range of pull_dates (e.g. 2000-01-01 to 2025-02-28) to simulate running the scripts on a daily basis.
- If the loop takes way too long, use a shorter range
- Compare the performance of each method (data consistency and speed)

### 4-1. Append load

In [ ]:
def simulation(start_date, end_date, db_file, excel_file):
    pull_dates = pd.date_range(start=start_date, end=end_date, freq='D').strftime('%Y-%m-%d')
    
    con = duckdb.connect(db_file)
    
    for pull_date in pull_dates:
        print(f"Processing data for {pull_date}")
        data = get_latest_data(excel_file, pull_date) 

        if data.empty:
            print(f"No data for {pull_date}, skipping...")
            continue  # Skip if no data


        con.sql('BEGIN TRANSACTION')
        
        #inserting append load
        append_load(con, data)
        
        con.sql('COMMIT')

    df = con.sql('SELECT date, cpi FROM data_append1').fetchdf()

    con.close()

    return df

simulation(start_date='2025-02-01',
            end_date='2025-02-28', 
            db_file = 'cpi.db',
            excel_file = 'CPI.xlsx')


Processing data for 2025-02-01
Processing data for 2025-02-02
Processing data for 2025-02-03
Processing data for 2025-02-04
Processing data for 2025-02-05
Processing data for 2025-02-06
Processing data for 2025-02-07
Processing data for 2025-02-08
Processing data for 2025-02-09
Processing data for 2025-02-10
Processing data for 2025-02-11
Processing data for 2025-02-12
Processing data for 2025-02-13
Processing data for 2025-02-14
Processing data for 2025-02-15
Processing data for 2025-02-16
Processing data for 2025-02-17
Processing data for 2025-02-18
Processing data for 2025-02-19
Processing data for 2025-02-20
Processing data for 2025-02-21
Processing data for 2025-02-22
Processing data for 2025-02-23
Processing data for 2025-02-24
Processing data for 2025-02-25
Processing data for 2025-02-26
Processing data for 2025-02-27
Processing data for 2025-02-28


,date,cpi
0,1947-01-01,21.480000
1,1947-02-01,21.620001
2,1947-03-01,22.000000
3,1947-04-01,22.000000
4,1947-05-01,21.950001
...,...,...
28105,2024-09-01,314.851013
28106,2024-10-01,315.563995
28107,2024-11-01,316.449005
28108,2024-12-01,317.602997


### 4-2. Truncate and load

In [22]:
# Giulio

### 4-3. Incremental load

In [6]:
def simulation(start_date, end_date, db_file, excel_file):
    
    # Generate date range
    pull_dates = pd.date_range(start=start_date, end=end_date, freq='D').strftime('%Y-%m-%d')
    
    # Initialize database
    con = duckdb.connect(db_file)
    
    for pull_date in pull_dates:
        print(f"Processing data for {pull_date}")
        data = get_latest_data(excel_file, pull_date) 

        # Start transaction
        con.sql('BEGIN TRANSACTION')
        incremental_load(con, data) # can be changed to append_load, truncate_load or incremental_load
        con.sql('COMMIT')

    # create data frame
    df = con.sql('SELECT * FROM data_incremental').fetchdf()

    # Close connection
    con.close()

    return df

# Run simulation
simulation(start_date='2025-02-01',
            end_date='2025-02-28', 
            db_file = 'cpi.db',
            excel_file = 'CPI.xlsx')

Processing data for 2025-02-01
Processing data for 2025-02-02
Processing data for 2025-02-03
Processing data for 2025-02-04
Processing data for 2025-02-05
Processing data for 2025-02-06
Processing data for 2025-02-07
Processing data for 2025-02-08
Processing data for 2025-02-09
Processing data for 2025-02-10
Processing data for 2025-02-11
Processing data for 2025-02-12
Processing data for 2025-02-13
Processing data for 2025-02-14
Processing data for 2025-02-15
Processing data for 2025-02-16
Processing data for 2025-02-17
Processing data for 2025-02-18
Processing data for 2025-02-19
Processing data for 2025-02-20
Processing data for 2025-02-21
Processing data for 2025-02-22
Processing data for 2025-02-23
Processing data for 2025-02-24
Processing data for 2025-02-25
Processing data for 2025-02-26
Processing data for 2025-02-27
Processing data for 2025-02-28


,date,cpi
0,1947-03-01,22.000000
1,1947-04-01,22.000000
2,1948-03-01,23.500000
3,1949-09-01,23.750000
4,1950-10-01,24.500000
...,...,...
1750,2024-09-01,314.851013
1751,2024-10-01,315.563995
1752,2024-11-01,316.449005
1753,2024-12-01,317.602997
